In [1]:
# Importing a dependancy
import pandas as pd

In [2]:
# Define the path for reading the csv
file = '../Resources/purchase_data.csv'

In [3]:
# Reading the csv file setting it to a dataframe "df"
df = pd.read_csv(file)

# Preview the data that will be used.
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# Create an empty dataframe
player_count_df = pd.DataFrame()

# Drop any duplicates to make the count easier
count_df = pd.DataFrame(df.drop_duplicates(subset = ['SN']))

In [5]:
# Perform a count function
players = count_df['SN'].count()

# Add to the dataframe
player_count_df['Total Players'] = [players]

# Show results of the dataframe
player_count_df

,Total Players
0,576


In [6]:
# Create a new dataframe for a summary table
summary_df = pd.DataFrame()

# Find the statistical measures
average = df['Price'].mean()
total = df['Price'].sum()
item_count = df['Item ID'].count()

# Keep only unique values
sub_df = df.drop_duplicates(subset = ['Item Name'])

# Count only the unique items
total_items = sub_df['Item Name'].count()

# Add all of the data to the existing dataframe
summary_df['Number of Unique Items'] = [total_items]
summary_df['Average Purchase Price'] = [average]
summary_df['Total Number of Purchases'] = [item_count]
summary_df['Total Revenue'] = [total]

# Format columns
summary_df['Average Purchase Price'] = summary_df['Average Purchase Price'].map("${:.2f}".format)
summary_df['Total Revenue'] = summary_df['Total Revenue'].map("${:.2f}".format)

#Show the dataframe
summary_df

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [52]:
##Comment later will use this code

gendertotal_df = count_df['Gender'].value_counts().rename_axis('Gender').reset_index(name = 'Total Count')
genderpercent_df = count_df['Gender'].value_counts('Male').rename_axis('Gender').reset_index(name = 'Percentage of Players')
gender_df = pd.merge(gendertotal_df, genderpercent_df, on = 'Gender')

gender_df['Percentage of Players'] = gender_df['Percentage of Players']*100
gender_df['Percentage of Players'] = gender_df['Percentage of Players'].map("{:.2f}%".format)

gender_df.set_index("Gender", inplace = True)

gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
# Find statistical measures under a certain group
gender_avg = df.groupby(['Gender'])['Price'].mean()
gender_count = df.groupby(['Gender'])['SN'].count()
gender_total = df.groupby(['Gender'])['Price'].sum()

# Merge two of the dataframes
gender_merge_df = pd.merge(gender_count, gender_avg, on = 'Gender')

# Appending the total price to the dataframe
gender_merge_df['Total Purchase Value'] = gender_total

# Find the count of all players but grouped by gender
gender_ind = count_df.groupby(['Gender'])['SN'].count()

# Find the average spending of gender groups
gender_average = gender_total / gender_ind

# Append to a new column in dataframe
gender_merge_df['Avg Total Purchase Per Player'] = gender_average

# Remame some columns
Gender_df = gender_merge_df.rename(columns={'SN':'Purchase Count', 'Price':"Average Purchase Price"})

# Format values to match their columns
Gender_df['Average Purchase Price'] = Gender_df['Average Purchase Price'].map("${:.2f}".format)
Gender_df['Total Purchase Value'] = Gender_df['Total Purchase Value'].map("${:.2f}".format)
Gender_df['Avg Total Purchase Per Player'] = Gender_df['Avg Total Purchase Per Player'].map("${:.2f}".format)

# Show the new dataframe
Gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase Per Player
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [9]:
# Created bins for age groups
bins = [0,9,14,19,24,29,34,39,48]

# Bin labels
group_names = ["< 10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

# Adds the new category "Age Group"
count_df["Age group"]= pd.cut(count_df["Age"], bins, labels=group_names, include_lowest=True)

# Creates a groupby dataframe using the reduced dataframe
age_df = count_df.groupby("Age group")

# Counts the amount of points within the given age group
unique_age_count = age_df["Age"].count()

# Places into a new dataframe
age_group_df = pd.DataFrame(unique_age_count)

# Renames a column and calculates percentages for a new column
age_group_df = age_group_df.rename(columns = {"Age":"Total Count"})
percent_age = (age_group_df / len(count_df)) * 100

# Adds a new column to the dataframe age_group_df
age_group_df["Percent of Players"] = percent_age

# Reformats the values in the new column
age_group_df["Percent of Players"] = age_group_df["Percent of Players"].map("{:.2f}%".format)

# Presents the dataframe
age_group_df

,Total Count,Percent of Players
Age group,,
< 10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [10]:
df["Age group"]= pd.cut(df["Age"], bins, labels=group_names, include_lowest=True)
summary_age_df = df.groupby("Age group")
purchase_count = summary_age_df["SN"].count()
summary_age_group_df = pd.DataFrame(purchase_count)
summary_age_group_df = summary_age_group_df.rename(columns = {"SN":"Purchase Count"})

summary_age_group_df["Average Purchase Price"] = summary_age_df["Price"].mean().map("$ {:.2f}".format)
summary_age_group_df["Total Purchase Value"] = summary_age_df["Price"].sum().map("${:.2f}".format)
total_age_amount = summary_age_df["Price"].sum()

avg_total_purchase = total_age_amount / unique_age_count
summary_age_group_df["Avg Total Purchase per Person"] = avg_total_purchase.map("${:.2f}".format)

summary_age_group_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age group,,,,
< 10,23,$ 3.35,$77.13,$4.54
10-14,28,$ 2.96,$82.78,$3.76
15-19,136,$ 3.04,$412.89,$3.86
20-24,365,$ 3.05,$1114.06,$4.32
25-29,101,$ 2.90,$293.00,$3.81
30-34,73,$ 2.93,$214.00,$4.12
35-39,41,$ 3.60,$147.67,$4.76
40+,13,$ 2.94,$38.24,$3.19


In [11]:
top_df = df.groupby("SN")
top_average_purchase = top_df["Price"].mean().map("${:.2f}".format)
top_total_purchase = top_df["Price"].sum().map("${:.2f}".format)

top_spender_df = pd.DataFrame(top_df["SN"].count())
top_spender_df["Average Purchase Price"] = top_average_purchase
top_spender_df["Total Purchase Value"] = top_total_purchase
top_spender_df = top_spender_df.rename(columns = {"SN":"Purchase Count"})

top_spender_df = top_spender_df.sort_values("Purchase Count", ascending = False)

top_spender_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


In [12]:
most_pop_item = df.groupby(["Item ID","Item Name"])

most_pop_count = most_pop_item["SN"].count()

most_pop_item_df = pd.DataFrame(most_pop_count)
most_pop_sum = most_pop_item["Price"].sum()
most_pop_price = most_pop_sum / most_pop_count

most_pop_item_df["Item Price"] = most_pop_price.astype(float).map("${:.2f}".format)
most_pop_item_df["Total Price"] = most_pop_sum.astype(float).map("${:.2f}".format)
most_pop_item_df = most_pop_item_df.rename(columns = {"SN":"Purchase Count"})

Most_pop_item_df = most_pop_item_df.sort_values("Purchase Count", ascending = False)
Most_pop_item_df.head()

,,Purchase Count,Item Price,Total Price
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [13]:
most_pop_item_df["Total Price"] = most_pop_item_df["Total Price"].str.replace("$", "", regex=True)
most_pop_item_df["Total Price"] = most_pop_item_df["Total Price"].astype("float")
most_prof_item_df = most_pop_item_df.sort_values("Total Price", ascending = False)
most_prof_item_df["Total Price"] = most_prof_item_df["Total Price"].map("${:.2f}".format)
most_prof_item_df.head()

,,Purchase Count,Item Price,Total Price
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


In [53]:
##Alternative to getting the count of genders references.


reduced_df = pd.DataFrame(df.loc[(df["SN"].value_counts())])
count1 = count_df['Gender'].value_counts()
count2 = r['Gender'] / len(reduced_df)
r = pd.DataFrame(count1)
r["Percentage of Players"] = count2 * 100
r["Percentage of Players"] = r["Percentage of Players"].map("{:.2f}%".format)
r = r.rename(columns = {"Gender":"Player Count"})

r